# Exercise 9- CIA 2

1. Implement a neural network from scratch. Use the gradient descent optimization technique
for weight optimization. Hint: you shall use make_classification function to create a new
dataset for classification problem/ use MNIST dataset or use the load_boston dataset for
regression problem.
2. For the same dataset, build a neural network using keras library. Run the same number of
epochs and compare the results obtained with your model vs the built-in keras model.

### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Data Generation- Classification

In [4]:
dataset_classification = make_classification(n_samples=1000, n_features=10, random_state=42)
X_class, y_class = dataset_classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)
scaler_class = StandardScaler()
X_train_class = scaler_class.fit_transform(X_train_class)
X_test_class = scaler_class.transform(X_test_class)

### Data Generation- Regression
Using the fetch_california_housing instead of load_boston as it is not present in the latest scikit version.

In [5]:
dataset_regression = fetch_california_housing()
X_reg, y_reg = dataset_regression.data, dataset_regression.target
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
scaler_reg = StandardScaler()
X_train_reg = scaler_reg.fit_transform(X_train_reg)
X_test_reg = scaler_reg.transform(X_test_reg)

### Training the Keras model and a neural network model built from scratch

In [17]:
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size, lr=0.05, batch_size=32):
        self.lr = lr
        self.batch_size = batch_size
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros((1, output_size))

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0).astype(float)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        m = y.shape[0]
        error = output - y.reshape(-1, 1)
        dW2 = np.dot(self.a1.T, error) / m
        db2 = np.sum(error) / m
        dW1 = np.dot(X.T, (np.dot(error, self.W2.T) * self.relu_derivative(self.a1))) / m
        db1 = np.sum((np.dot(error, self.W2.T) * self.relu_derivative(self.a1)), axis=0) / m

        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            for i in range(0, X.shape[0], self.batch_size):
                X_batch = X[i:i+self.batch_size]
                y_batch = y[i:i+self.batch_size]
                output = self.forward(X_batch)
                self.backward(X_batch, y_batch, output)

            if epoch % 10 == 0:
                loss = np.mean((self.forward(X) - y.reshape(-1, 1)) ** 2)
                print(f"Epoch {epoch}: Loss {loss}")

# Training the neural network model built from scratch
nn_class = SimpleNN(input_size=10, hidden_size=10, output_size=1, lr=0.05)
nn_class.train(X_train_class, y_train_class, epochs=1000)

#Training the Keras model
model_class = Sequential([
    Dense(32, activation='relu', input_shape=(10,)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_class.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_class.fit(X_train_class, y_train_class, epochs=1000, verbose=2, batch_size=32)

Epoch 0: Loss 0.16883899892430995
Epoch 10: Loss 0.09595416974018509
Epoch 20: Loss 0.08958849708551313
Epoch 30: Loss 0.0880535549496389
Epoch 40: Loss 0.08713370051107422
Epoch 50: Loss 0.08616418031902694
Epoch 60: Loss 0.08522627994685841
Epoch 70: Loss 0.08434828679437124
Epoch 80: Loss 0.0835785833060794
Epoch 90: Loss 0.08285725230762338
Epoch 100: Loss 0.08225877578848845
Epoch 110: Loss 0.08183359651412918
Epoch 120: Loss 0.0813880962254447
Epoch 130: Loss 0.0809370498364822
Epoch 140: Loss 0.0804429467163498
Epoch 150: Loss 0.07993826187928713
Epoch 160: Loss 0.07947528383034157
Epoch 170: Loss 0.07910192603986375
Epoch 180: Loss 0.0787350968614155
Epoch 190: Loss 0.07834070067994185
Epoch 200: Loss 0.07776908213949153
Epoch 210: Loss 0.07714146867736198
Epoch 220: Loss 0.07645680929910248
Epoch 230: Loss 0.07594852502944667
Epoch 240: Loss 0.07530897736560137
Epoch 250: Loss 0.07468174346934134
Epoch 260: Loss 0.07418971601503013
Epoch 270: Loss 0.07354743466794264
Epoch 280

### Training the keras model for Regression

In [13]:
model_reg = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_reg.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
model_reg.compile(optimizer='adam', loss='mean_squared_error')
model_reg.fit(X_train_reg, y_train_reg, epochs=100, verbose=2, batch_size=32)

Epoch 1/100
516/516 - 5s - 9ms/step - loss: 0.7698
Epoch 2/100
516/516 - 1s - 2ms/step - loss: 0.5124
Epoch 3/100
516/516 - 1s - 2ms/step - loss: 0.4639
Epoch 4/100
516/516 - 1s - 2ms/step - loss: 0.4271
Epoch 5/100
516/516 - 2s - 3ms/step - loss: 0.4153
Epoch 6/100
516/516 - 1s - 3ms/step - loss: 0.3951
Epoch 7/100
516/516 - 3s - 5ms/step - loss: 0.3933
Epoch 8/100
516/516 - 3s - 5ms/step - loss: 0.3819
Epoch 9/100
516/516 - 2s - 4ms/step - loss: 0.3714
Epoch 10/100
516/516 - 2s - 3ms/step - loss: 0.3633
Epoch 11/100
516/516 - 2s - 4ms/step - loss: 0.3596
Epoch 12/100
516/516 - 1s - 3ms/step - loss: 0.3531
Epoch 13/100
516/516 - 2s - 4ms/step - loss: 0.3506
Epoch 14/100
516/516 - 2s - 4ms/step - loss: 0.3426
Epoch 15/100
516/516 - 1s - 2ms/step - loss: 0.3418
Epoch 16/100
516/516 - 1s - 2ms/step - loss: 0.3433
Epoch 17/100
516/516 - 1s - 2ms/step - loss: 0.3355
Epoch 18/100
516/516 - 1s - 2ms/step - loss: 0.3308
Epoch 19/100
516/516 - 1s - 2ms/step - loss: 0.3288
Epoch 20/100
516/516 

### Evaluvation for the Classification

In [18]:
nn_preds_class = nn_class.forward(X_test_class).flatten()
nn_preds_class = np.where(nn_preds_class > 0.5, 1, 0)
keras_preds_class = model_class.predict(X_test_class).flatten()
keras_preds_class = np.where(keras_preds_class > 0.5, 1, 0)

accuracy_nn_class = np.mean(nn_preds_class == y_test_class)
accuracy_keras_class = np.mean(keras_preds_class == y_test_class)

print(f"Accuracy of Neural Network from Scratch (Classification): {accuracy_nn_class}")
print(f"Accuracy of Keras Model (Classification): {accuracy_keras_class}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Accuracy of Neural Network from Scratch (Classification): 0.915
Accuracy of Keras Model (Classification): 0.865


### Evaluvation using MSE for regression.

In [14]:
mse_keras_reg = model_reg.evaluate(X_test_reg, y_test_reg)
print(f"Mean Squared Error of Keras Regression Model: {mse_keras_reg}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2791
Mean Squared Error of Keras Regression Model: 0.2917464077472687


### Achieved an accuracy of 91.5% for the self built neural network model and an accuracy of 86.5% for the keras model for classification and a mean squared error of 0.29 for the regression model.

### Key Inferences: Neural Network vs Keras Model

    1. Accuracy & Stability: Keras achieves higher accuracy and smoother training due to better optimization.
    2. Optimization: Adam optimizer in Keras dynamically adjusts learning rates, unlike fixed gradient descent.
    3. Regularization: Keras uses Batch Normalization & Dropout to prevent overfitting, which the manual model lacks.
    4. Training Efficiency: Keras handles complex architectures efficiently, while manual implementation requires careful tuning.
    5. Ease of Use: Manual models offer customization but require more effort, whereas Keras simplifies model building.